# Value Maximisation with Genetic Algorithms 🧬

---

## Summary

- [1. Problem Introduction](#problem-introduction)
- [2. Genetic Algorithms Presentation](#genetic-algorithms-presentation)
- [3. Problem Implementation](#problem-implementation)
- [4. Resolution and Fitness Comparison](#resolution-and-fitness-comparison)
- [5. Is Genetic Algorithms Good for This Problem?](#is-genetic-algorithms-good-for-this-problem)

---
<a name="problem-introduction"></a>
## 1. Problem Introduction

### Biscuit manufacturing factory problem 🍪
A biscuit manufacturing factory is gearing up for the festive season by producing a variety of biscuits. The challenge? Maximising biscuit production and profit from a single roll of dough. Here’s the breakdown:

### Key Information: 
- **Dough Roll Properties**:
  - A predefined rectangular length, denoted as **‘LENGTH’** (one-dimensional problem).
  - Contains **defects** at specific positions (**x**) and of various types (**a**, **b**, **c**, etc.).

- **Biscuits**:
  - Can be produced in infinite quantities.
  - Have specific **sizes**, **values**, and **defect thresholds** (maximum allowable defects of each class).

### Constraints: 
1. **Placement Rules**:
   - Biscuits must be placed at **integer positions**.
   - **No overlapping**: Positions occupied by one biscuit cannot be used by another.
2. **Defect Tolerance**:
   - A biscuit’s defect limits must not be exceeded for the positions it covers.
3. **Length Limitation**:
   - The total size of the biscuits cannot exceed the dough roll’s length.

### Solution Value: 
- The value of a solution = **sum of biscuit values** – **penalty for unused dough** (-1 per empty position).

### Project Benchmark:
- **Roll Length**: 500 units.
- **Defects**: 3 classes (a, b, c), details in `defects.csv`.
- **Biscuits**:  

| Biscuit   | Length | Value | Defects (a, b, c) |
|-----------|--------|-------|-------------------|
| Biscuit 0 | 4      | 3     | (4, 2, 3)        |
| Biscuit 1 | 8      | 12    | (5, 4, 4)        |
| Biscuit 2 | 2      | 1     | (1, 2, 1)        |
| Biscuit 3 | 5      | 8     | (2, 3, 2)        |

---
<a name="genetic-algorithms-presentation"></a>
## 2. Genetic Algorithms Presentation 🧬🔄

### What are Genetic Algorithms (GAs)?
Genetic Algorithms (GAs) are optimisation techniques inspired by **natural selection**. They simulate the process of evolution to find solutions to complex problems. GAs are particularly effective for large search spaces and problems with multiple constraints.

### Key Steps in GAs:
1. **Initialisation** 🔄:
   - Start with a randomly generated population of potential solutions.

2. **Evaluation** 💯:
   - Calculate the fitness of each solution (e.g., total value of biscuits – penalties).

3. **Selection** 🥇:
   - Choose the best solutions based on fitness to form the next generation.

4. **Crossover** 🔗:
   - Combine parts of two solutions (parents) to produce new solutions (children).

5. **Mutation** 🍀:
   - Introduce small changes to solutions to explore new possibilities.

6. **Termination** ⏳:
   - Stop when a satisfactory solution is found or after a predefined number of generations.

---
<a name="problem-implementation"></a>
## 3. Problem Implementation ⚙️


In [1]:
import numpy as np
import random
import pandas as pd

1. **Initialisation** 🔄  
An individual is made up of a list the size of the roll. Each item corresponds to a piece of biscuit placed at a position on the roll.

In [2]:
def init_population(pop_size, roll_size):
    
    return [np.random.randint(-1, 4, size=(roll_size,)) for _ in range(pop_size)]

2. **Evaluation** 💯   
We'll start by defining a classic evaluation method based on our constraints

In [3]:
# Short method to check if a biscuit can be put in a certain position
def respect_defects(threshold, start, end, roll_defects):
    pos_defects = {"a": 0, "b": 0, "c": 0}
    for pos in range(start, end):
        for key in roll_defects[pos]:
            pos_defects[key] += (roll_defects[pos][key])
    return all(threshold[key] >= pos_defects[key] for key in pos_defects.keys())

In [4]:
# Basic fitness method, each whole biscuit adds its value to the total, and each position with a non-whole biscuit loses -1.
def fitness_1(ind, biscuits_list, roll_defects):
    
    score = 0
    size = 0
    last_elem = -2  # So we don't have to check if we test the first value of the list each iteration doing ind[i-1]
    for i, elem in enumerate(ind):
        if elem == -1:  # If "no biscuit value" no need to test anymore
            score -= (1 + size)
            size = 0
        else:
            if elem != last_elem:
                score -= size
                size = 1
            else:
                size += 1
            if size == biscuits_list[elem]["size"]:  # Test the defects only if the biscuit reach its required size
                if respect_defects(biscuits_list[elem]["threshold"], i - size + 1, i + 1, roll_defects):
                    score += biscuits_list[elem]["value"]
                    size = 0
                else:
                    score -= 1
                    size -= 1
        last_elem = elem
    score -= size  # Don't forget to remove last biscuit pieces that are not whole
    return score

The previous method can be a little simple to achieve good results. We can implement a second method that gives more weight to biscuits that are almost whole (instead of subtracting -1 for each piece).  
For example, a type 3 biscuit have a size of 5 and a value of 8. In our first method, if only 3 pieces of biscuit 3 are aligned, they will have a score of -3. In this method, they will have a score of 8 * 3/5^2.  
**The main formula will be:**  
(total value) * (proportion of biscuit)^2 

In [5]:
# Returns the score of a slice of 'size' number of biscuit with the same type 
def get_slice_score(position, size, lb_size, lb_value, lb_threshold, roll_defects):  # lb stands for last biscuit
    score = 0
    unassigned = [i for i in range(position - size, position)]  # Ensure we give a value to every biscuit of the last biscuit type 
    last_size = min(lb_size, size)
    while unassigned and last_size != 0:  # While we don't have assigned a value for every last position
        j = 0
        assigned = []
        while j <= len(unassigned) - last_size:
            pos = unassigned[j]
            start = pos  # Start of the continuation of the previous value
            end = pos + last_size
            if all(elem in unassigned for elem in range(start, end)):
                if respect_defects(lb_threshold, start, end, roll_defects):
                    score += (last_size/lb_size)**2 * lb_value  # the closer the biscuit is to its full size, the more importance is given to it
                    for rem in range(start, end):
                        assigned.append(rem)
                    j += last_size - 1  # Put -1 because there is a j+=1 at the end of the loop anw 
            j+=1
        for assi in  assigned:
            unassigned.remove(assi) 
        last_size -= 1
    score -= len(unassigned)  # -1 For all element that do not respect defects even alone
    return score

In [6]:
# fitness method that give more importants to biscuits that aren't full but almost
def fitness_2(ind, biscuits_list, roll_defects):
    
    score = 0
    size = 0
    last_elem = ind[0]  # So we don't have to check if we test the first value of the list each iteration doing ind[i-1]
    for i, elem in enumerate(ind):
        if elem == last_elem:
            size += 1
        else:
            if last_elem == -1:
                score -= size
            else:
                slice_score = get_slice_score(i, size, biscuits_list[last_elem]["size"], biscuits_list[last_elem]["value"], biscuits_list[last_elem]["threshold"], roll_defects)
                score += slice_score
            size = 1
        last_elem = elem
    score += get_slice_score(len(ind), size, biscuits_list[last_elem]["size"], biscuits_list[last_elem]["value"], biscuits_list[last_elem]["threshold"], roll_defects)
                
    return score

3. **Selection** 🥇  
- Elite selection:  
We'll start by selecting an **‘elite_ratio’ percentage** of our population containing the individuals with the best score. These individuals will not undergo any modifications and will go directly to the next generation.

- Parent selection:  
Next, we need to add new individuals to our population. To do this, we will randomly select two parents, each of whom will produce one child. The choice of parents will be based on their score; the higher the score of an individual, the greater the chance that they will be chosen as parents. This allows individuals with a low score to still have a chance of being chosen and maintains genetic diversity over the generations.

4. **Crossover** 🔗  
- For the cross over, we will choose 3 break points at random between 0 and 499 (all different) bp1, bp2 and bp3. And the child will be a random mix of 4 pieces from one of its parents each time:  
**0:bp1 + bp1:bp2 + bp2:bp3 + bp3:499** 
This will allow us to randomly mix the two parents without breaking too many whole biscuits.

5. **Mutation** 🍀  
- Mutations will be applied to each of the child's positions separately, with each position having a **‘mutation_rate’% chance** of becoming a new random biscuit type between -1 and 3. 

In [7]:
def evolve_pop(population, mutation_rate, elite_ratio, biscuits_list, roll_defects, fitness):

    # Compute probability for each individual
    fitness_values = [fitness(ind, biscuits_list, roll_defects) for ind in population]

    # Chose elites based on individual fitness scores
    elite_idx = np.argsort(fitness_values)[int(-len(population) * elite_ratio):]
    
    elites = [population[i] for i in elite_idx]

    min_fitness = min(fitness_values)
    shifted_fitness = [f - min_fitness + 1 for f in fitness_values]  # Add 1 to avoid negative fitness for porbabilities compute

    fitness_sum = sum(shifted_fitness)
    probabilities = [f / fitness_sum for f in shifted_fitness]
    

    new_population = []
    while len(new_population) < len(population) - len(elites):
        
        # Cross over
        # Select two parents based on fitness probabilities
        parents_indices = np.random.choice(len(population), size=2, replace=False, p=probabilities)
        parent1 = population[parents_indices[0]]
        parent2 = population[parents_indices[1]]

        child = np.zeros(parent1.shape)
        break_points = np.random.choice(len(parent1), size=3, replace=False)
        break_points = np.insert(break_points, 0, [0, len(parent1)])
        break_points.sort()
        for i in range(len(break_points)-1):
            start = break_points[i]
            end = break_points[i+1]
            chosen_p = parent1 if random.random() < 0.5 else parent2
            child[start:end] = chosen_p[start:end]

        # Mutation
        muted_child = np.array([gene if random.random() > mutation_rate else random.randint(-1, 3) for gene in child] )

        new_population.append(muted_child)
        
    return new_population + elites

In [8]:
def genetic_algorithm(pop_size, mutation_rate, elite_ratio, biscuits_list, roll_defects, roll_size, max_iter, display, fitness, population=None, rtype="elite"):
    if population is None:
        population = init_population(pop_size, roll_size)
    
    for i in range(max_iter):
        population = evolve_pop(population, mutation_rate, elite_ratio, biscuits_list, roll_defects, fitness)
        
        if display and (i+1) % display == 0:
            # Metric computation, remove 4 lines below to go faster
            fitness_values = [fitness(ind, biscuits_list, roll_defects) for ind in population]
            elite_idx = np.argsort(fitness_values)[-1]
            print(f'Generation {i+1}: Best fitness {fitness_values[elite_idx]}')

    if rtype != "population":  # if rtype = "population" we want to return the entire population for further improves
        fitness_values = [fitness(ind, biscuits_list, roll_defects) for ind in population]
        elite_idx = np.argsort(fitness_values)[-1]
        population = [population[elite_idx]]  # returns only the best element
    
    return population
    

---
<a name="resolution-and-fitness-comparison"></a>
## 4. Resolution and Fitness Comparison 📊

First, let's load defects and initialize our biscuits

In [9]:
df = pd.read_csv("defects.csv")
print(df.shape)
df = df.sort_values(by="x")
df.head(1)

(500, 2)


,x,class
479,0.700561,a


In [10]:
# dict format -> id : (value, size, defects_threshold)

biscuits_list = {
    -1: ({"value": -1, "size": 1, "threshold": {"a":9, "b":9, "c":9}}),
     0: ({"value":  3, "size": 4, "threshold": {"a":4, "b":2, "c":3}}),
     1: ({"value": 12, "size": 8, "threshold": {"a":5, "b":4, "c":4}}),
     2: ({"value":  1, "size": 2, "threshold": {"a":1, "b":2, "c":1}}),
     3: ({"value":  8, "size": 5, "threshold": {"a":4, "b":2, "c":3}}),
}

In [11]:
roll_size = 500
roll_defects = {i: {"a": 0, "b": 0, "c": 0} for i in range(roll_size)}
for _, row in df.iterrows():
    if int(row["x"]) >= roll_size:
        break
    roll_defects[int(row["x"])][row["class"]] += 1

Now let's try different mutation rates with the fitness_2 method and see what works better

In [13]:
pop_size = 100
elite_ratio = 0.1
max_iter = 1000

In [16]:
for mutation_rate in [0.01, 0.02, 0.03, 0.04, 0.1]:
    [result] = genetic_algorithm(pop_size, mutation_rate, elite_ratio, biscuits_list, roll_defects, roll_size, max_iter, None, fitness_2)
    print(f"m_rate: {mutation_rate} best fit score 1: {fitness_1(result, biscuits_list, roll_defects)} | best fit score 2: {fitness_2(result, biscuits_list, roll_defects)}")

m_rate: 0.01 best fit score 1: 420 | best fit score 2: 645.0124999999999
m_rate: 0.02 best fit score 1: 281 | best fit score 2: 560.6774999999999
m_rate: 0.03 best fit score 1: 64 | best fit score 2: 463.4299999999999
m_rate: 0.04 best fit score 1: -14 | best fit score 2: 419.1599999999997
m_rate: 0.1 best fit score 1: -301 | best fit score 2: 258.6799999999998


Obviously the best mutation rate is 0.01, other are to high and probably cut too many already-formed biscuits.  
We also display the score given by the fitness method, since this is the true score given by our biscuits, not counting those that are not whole.

In [15]:
pop_size = 1000
mutation_rate = 0.01
max_iter = 1000

In [16]:
for elite_ratio in [0.01, 0.02, 0.05, 0.1, 0.2]:
    [result] = genetic_algorithm(pop_size, mutation_rate, elite_ratio, biscuits_list, roll_defects, roll_size, max_iter, None, fitness_2)
    print(f"elite_ratio: {elite_ratio} best fit score 1: {fitness_1(result, biscuits_list, roll_defects)} | best fit score 2: {fitness_2(result, biscuits_list, roll_defects)}")

elite_ratio: 0.01 best fit score 1: 62 | best fit score 2: 469.5399999999998
elite_ratio: 0.02 best fit score 1: 156 | best fit score 2: 532.6224999999997
elite_ratio: 0.05 best fit score 1: 350 | best fit score 2: 626.835
elite_ratio: 0.1 best fit score 1: 396 | best fit score 2: 642.9
elite_ratio: 0.2 best fit score 1: 451 | best fit score 2: 646.5325


Now that we've found the best parameters for fitness_2, we can try comparing it with fitness to see if there's a difference.

In [17]:
# Use the same parameters because it is the same problem
pop_size = 1000
mutation_rate = 0.01
elite_ratio = 0.2
max_iter = 1000
display = max_iter // 5

In [18]:
[elite] = genetic_algorithm(pop_size, mutation_rate, elite_ratio, biscuits_list, roll_defects, roll_size, max_iter, display, fitness_1)

Generation 200: Best fitness 120
Generation 400: Best fitness 174
Generation 600: Best fitness 207
Generation 800: Best fitness 225
Generation 1000: Best fitness 231


Results obtained are really bad compare to fitness_2

The fitness2 method makes it possible to construct biscuits with a greater length, as their pieces do not reduce the total value. We can see that this method achieves better values than fitness. On the other hand, fitness removes non-whole biscuits too quickly because they are counted as malus, but this is a good thing for our final result because we only want whole biscuits. We can try to see if by combining one and then the other we can get a better result:
- Start with fitness_2 to obtain large biscuits 
- We continue with the same population with fitness to try and complete these biscuits and remove the pieces that are too small. 

In [19]:
# Keep the parameters we found before but now save the entire poplation
pop_size = 1000
mutation_rate = 0.01
elite_ratio = 0.2
max_iter = 1000
display = max_iter // 5
population = genetic_algorithm(pop_size, mutation_rate, elite_ratio, biscuits_list, roll_defects, roll_size, max_iter, display, fitness_2, rtype="population")

Generation 200: Best fitness 491.8399999999998
Generation 400: Best fitness 580.4825000000001
Generation 600: Best fitness 622.0500000000001
Generation 800: Best fitness 648.5475
Generation 1000: Best fitness 656.3750000000001


Now let's give this poulation to the GA with fitness_1 and see if we improve the score

In [20]:
pop_size = len(population)
mutation_rate = 0.01
elite_ratio = 0.2
max_iter = 1000
display = max_iter // 5
elite = genetic_algorithm(pop_size, mutation_rate, elite_ratio, biscuits_list, roll_defects, roll_size, max_iter, display, fitness_1, population=population)

Generation 200: Best fitness 479
Generation 400: Best fitness 492
Generation 600: Best fitness 501
Generation 800: Best fitness 510
Generation 1000: Best fitness 519


This combination gives a maximum value of 519, which is much higher than the results for each of them separately. But we obtained a better score and in a much shorter time with the CSP, so we can wonder whether genetic algorithms are a good choice for solving this problem.

---
<a name="is-genetic-algorithms-good-for-this-problem"></a>
## 5. Is Genetic Algorithms Good for This Problem? ❔🕵️‍♂️

**This problem is a value optimisation problem with a very large set of possible solutions.**  
Genetic algorithms can be a relevant choice for solving it, since they can find a fast and optimised solution to problems with a large number of possible solutions.  

However, one detail makes modelling this problem in the form of a genetic algorithm complex:  

- **The biscuits have different sizes:**  
  In fact, this prevents us from placing only whole biscuits on our roll during initialisation, cross-over and mutation. Why?  
  1. **There's a risk that the biscuits will break during cross over**, as there's a good chance that the two parents won't be cut between two biscuits to create their child.  
  2. **The same goes for mutations.** How can you randomly replace one biscuit with another when they are not the same size?  

For these two reasons, we preferred to use biscuit pieces, but this only limited the efficiency of the algorithm to a large extent:  

- There are many more possibilities, which makes calculation times longer.  
- The algorithm has difficulty in making whole biscuits despite the fitness method, which leads to a large loss of value.